In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('Reddit_Encoded.csv')
df

,comment,hate_speech,lemmatized_comment,document_vector_flat
0,subsection retarded hungarians ohh boy brace l...,1,subsection retard hungarians ohh boy brace liv...,"0.014043219,-0.01809359,0.017145459,0.08062436..."
1,hiii just got work Foundation and grounding ma...,0,hiii just get work Foundation and ground mainl...,"-0.0030388932,-0.035133556,0.020659983,0.07383..."
2,wow guess soyboys every country,0,wow guess soyboys every country,"0.017362628,0.005587179,0.0297773,0.109146975,..."
3,owen benjamins soyboy song goes every country ...,0,owen benjamins soyboy song go every country amaze,"0.018085241,0.0011954829,2.8959475e-05,0.07601..."
4,yall hear sumn means live small town rn for w...,0,yall hear sumn mean live small town rn for wor...,"0.023993038,-0.00060867134,0.005239945,0.05989..."
...,...,...,...,...
22206,op stop faggot post videos next time hard,1,op stop faggot post videos next time hard,"0.08339707,-0.017676119,-0.036872935,0.1038081..."
22207,minute long video top hate champagne goes need...,0,minute long video top hate champagne go need g...,"0.009342635,0.02728245,-0.0013451587,0.0765217..."
22208,clue whos ecelebs are point time need get alo...,1,clue whos ecelebs be point time need get along...,"-0.0006810841,0.008315975,0.029024707,0.077029..."
22209,didn’t insult you insult me,0,didn ’ t insult you insult me,"0.06474433,-0.17082486,-0.029483724,0.04382453..."


In [8]:
df['fasttext_vector'] = df['document_vector_flat'].apply(lambda x: np.fromstring(x.strip('[]'), sep=','))
X = np.vstack(df['fasttext_vector'].values)
y=df['hate_speech']

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

### SMOTE

In [10]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority',random_state=42)
X_sm, y_sm = smote.fit_resample(X_train,y_train)
y_sm.value_counts()

0    13539
1    13539
Name: hate_speech, dtype: int64

### Applying Logistic Regression

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_score,recall_score,f1_score
lr = LogisticRegression(solver='sag',multi_class='ovr')

In [37]:
lr.fit(X_sm,y_sm)
y_pred_lr=lr.predict(X_test)

In [38]:
print("Logistic Regression Performancbe:")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))

Logistic Regression Performancbe:
Confusion Matrix:
 [[2799  587]
 [ 293  764]]
              precision    recall  f1-score   support

           0       0.91      0.83      0.86      3386
           1       0.57      0.72      0.63      1057

    accuracy                           0.80      4443
   macro avg       0.74      0.77      0.75      4443
weighted avg       0.82      0.80      0.81      4443



In [39]:
accuracy=accuracy_score(y_test,y_pred_lr)
precision=precision_score(y_test,y_pred_lr)
recall=recall_score(y_test,y_pred_lr)
f1=f1_score(y_test,y_pred_lr)
print(f"Accuracy:{accuracy:.2f}")
print(f"precision:{precision:.2f}")
print(f"recall:{recall:.2f}")
print(f"f1-score:{f1:.2f}")

Accuracy:0.80
precision:0.57
recall:0.72
f1-score:0.63


### Applying hyperparameter tuning for Logistic Regression

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV 

In [40]:
param_grid = [    
    {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter' : [100, 1000,2500, 5000]
    }
]

In [41]:
clf = GridSearchCV(lr, param_grid = param_grid,n_jobs=-1,cv = 3, verbose=10)

In [43]:
best_clf = clf.fit(X,y)

Fitting 3 folds for each of 1600 candidates, totalling 4800 fits


KeyboardInterrupt: 

In [ ]:
best_clf.best_estimator_

In [26]:
random_search = RandomizedSearchCV(LogisticRegression(), 
                                   param_grid)
random_search.fit(X_train, y_train) 
print(random_search.best_estimator_) 

C:\Users\moham\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\moham\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\moham\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\moham\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\moham\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\moham\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='no

LogisticRegression(C=78.47599703514607, max_iter=1000, penalty='l1',
                   solver='liblinear')


In [27]:
lr_random=LogisticRegression(C=78.47599703514607,max_iter=1000, penalty='l1',solver='liblinear')


In [29]:
lr_random.fit(X_sm,y_sm)

LogisticRegression(C=78.47599703514607, max_iter=1000, penalty='l1',
                   solver='liblinear')

In [32]:
y_pred_lr_r=lr_random.predict(X_test)

In [33]:
print("Logistic Regression Performance:")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lr_r))
print(classification_report(y_test, y_pred_lr_r))

Logistic Regression Performance:
Confusion Matrix:
 [[2766  620]
 [ 271  786]]
              precision    recall  f1-score   support

           0       0.91      0.82      0.86      3386
           1       0.56      0.74      0.64      1057

    accuracy                           0.80      4443
   macro avg       0.73      0.78      0.75      4443
weighted avg       0.83      0.80      0.81      4443



In [34]:
accuracy=accuracy_score(y_test,y_pred_lr_r)
precision=precision_score(y_test,y_pred_lr_r)
recall=recall_score(y_test,y_pred_lr_r)
f1=f1_score(y_test,y_pred_lr_r)
print(f"Accuracy:{accuracy:.2f}")
print(f"precision:{precision:.2f}")
print(f"recall:{recall:.2f}")
print(f"f1-score:{f1:.2f}")

Accuracy:0.80
precision:0.56
recall:0.74
f1-score:0.64
